# Lab07 Пример решения с SQL

### Задача

По имеющимся данным портала eclass.cc построить content-based рекомендации по образовательным курсам.

In [1]:
my_courses = [[8150, u'en', u'StatLearning: Statistical Learning'], 
              [25679, u'en', u'Video Lighting Basics - Udemy'], 
              [7791, u'es', u'Programaci\xf3n CNC - Fresadoras'], 
              [23111, u'es', u'C\xf3mo Crear un Blog Gratis en Google Blogger - Udemy'], 
              [1396, u'ru', u'\u0412\u0432\u0435\u0434\u0435\u043d\u0438\u0435 \u0432\u043e \u0432\u0441\u0442\u0440\u043e\u0435\u043d\u043d\u044b\u0435 \u0441\u0438\u0441\u0442\u0435\u043c\u044b \u0438 Windows Embedded CE'], 
              [1348, u'ru', u'\u0422\u0435\u0445\u043d\u043e\u043b\u043e\u0433\u0438\u044f Microsoft ADO .NET']
             ]

In [2]:
ids = [i[0] for i in my_courses]
ids

[8150, 25679, 7791, 23111, 1396, 1348]

In [3]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [4]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [5]:
%%time

df = spark.read.json("/labs/lab07data/DO_record_per_line.json")
df

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 9.07 s


In [6]:
df.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)



In [7]:
df.show(10)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|What’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|The goal of the D...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [8]:
docs = df.filter((df.lang == 'en') | (df.lang == 'es') | (df.lang == 'ru'))

In [9]:
docs.count()

27158

### добавляем колонку со словами

In [10]:
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType

In [11]:
docs.registerTempTable('courses')
sqlContext.registerFunction('tokenizer', lambda x: x.split(), ArrayType(StringType()))
wordsData = spark.sql('SELECT *, tokenizer(desc) as words FROM courses')

In [12]:
wordsData.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|               words|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[This, course, in...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[This, online, co...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[We, live, in, a,...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|[This, self-paced...|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|[This, game-based...|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
only showi

### строим модель

In [13]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer

tokenizer = RegexTokenizer(inputCol="desc", outputCol="words", gaps=False, 
                           pattern=r"[a-zA-Zа-яА-ЯёáéíóúüñÑýÁÉÍÓÚÝ0-9_\-]{2,}")
wordsData = tokenizer.transform(docs)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=6000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.show(10)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|               words|         rawFeatures|            features|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|(6000,[36,63,92,1...|(6000,[36,63,92,1...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|(6000,[3,128,160,...|(6000,[3,128,160,...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, di...|(6000,[13,200,237...|(6000,[13,200,237...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|

### нормализуем

In [14]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=2.0)
normalizedData = normalizer.transform(rescaledData)

In [15]:
normalizedData.show(10)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|               words|         rawFeatures|            features|        normFeatures|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|(6000,[36,63,92,1...|(6000,[36,63,92,1...|(6000,[36,63,92,1...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|(6000,[3,128,160,...|(6000,[3,128,160,...|(6000,[3,128,160,...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, di...|(6000,[13,200,237...|(6000

### считаем косинусную меру

In [16]:
from pyspark.mllib.linalg import Vectors, Matrices
def cosine_similarity(a,b):
    dot = a.dot(b)
    return str(dot)

In [17]:
from tqdm import tqdm
from pyspark.sql.types import FloatType

normalizedData.registerTempTable('courses')

result = {}

for i in tqdm(ids):
    chosen_doc = spark.sql('SELECT * FROM courses WHERE id = %i' % i).first()
    
    sqlContext.registerFunction('cosine_similarity',
            lambda x: cosine_similarity(x, chosen_doc.normFeatures)
    )
    
    resultData = spark.sql(
        '''
        SELECT id, name, lang, cosine_similarity(normFeatures) as similarity 
        FROM courses 
        WHERE lang = "%s" 
        ORDER BY similarity DESC, name, id
        ''' % chosen_doc.lang
    )

    result[str(i)] = [r.id for r in resultData[['id']].take(11)[1:11]]

100%|██████████| 6/6 [00:23<00:00,  3.98s/it]


In [18]:
print(result)

{'8150': [13273, 8145, 16837, 22411, 8306, 26907, 8146, 540, 8142, 19887], '25679': [7297, 4466, 24891, 4799, 4290, 5019, 8588, 6243, 6129, 6161], '7791': [21853, 387, 10035, 21107, 11474, 386, 4096, 26336, 25008, 26670], '23111': [9285, 13224, 6864, 9352, 9286, 19404, 10668, 26336, 26670, 23118], '1396': [1006, 20314, 8215, 1347, 20102, 17015, 22221, 955, 1375, 8792], '1348': [1257, 819, 20307, 823, 829, 1285, 963, 20312, 1229, 1011]}


In [19]:
import json
json.dump(result, open('lab07s.json', 'w'), sort_keys=True)